In [1]:
import pandas as pd
pd.set_option('display.max_columns',None)

import sys
sys.path.append("../..")  # 상위 디렉토리를 sys.path에 추가

import requests
import pandas as pd
from mySetting import Client_ID, Client_Secret

# 네이버 API 키
client_id = Client_ID  # 클라이언트 ID
client_secret = Client_Secret  # 클라이언트 Secret

def get_coordinates(address):
    url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
    headers = {
        'X-NCP-APIGW-API-KEY-ID': client_id,
        'X-NCP-APIGW-API-KEY': client_secret,
    }
    params = {
        'query': address,
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if data['addresses']:
        x = data['addresses'][0]['x']
        y = data['addresses'][0]['y']
        return (x, y)
    else:
        return (None, None)

In [4]:
def get_apart(new_apart_excel, apart_csv):
    new_apart = pd.read_excel('../../' + new_apart_excel, skiprows=12)
    new_apart['주소'] = new_apart['시군구'] + ' ' + new_apart['번지']
    new_apart = new_apart.reset_index()[['단지명','주소','층','계약년월','전용면적(㎡)','건축년도','거래금액(만원)']]
    new_apart['위도'] = pd.NA
    new_apart['경도'] = pd.NA

    for i, address in enumerate(new_apart['주소']):
        lng, lat = get_coordinates(address)
        new_apart.at[i, '위도'] = lat
        new_apart.at[i, '경도'] = lng

    new_apart.dropna(inplace=True)

    apart = pd.read_csv('../../' + apart_csv)

    apart = pd.concat([apart, new_apart])

    return apart

In [5]:
new_apart_excel = 'apart_data/아파트_실거래가_2021_2022.xlsx'
apart_csv = 'preprocessed/아파트거래정보/아파트.csv'

new_apart = get_apart(new_apart_excel, apart_csv)

/Users/hdk/anaconda3/envs/qa_system/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
new_apart.to_csv('../../preprocessed/아파트거래정보/아파트.csv', index=False)